In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- 1. Data Cleaning & Loading Functions ---
def clean_waiting_time(val):
    if pd.isna(val): return np.nan
    if isinstance(val, str):
        val = val.strip()
        try:
            if ':' in val:
                parts = val.split(':')
                return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])
            if '.' in val:
                parts = val.split('.')
                # Handle formats like 00.00.20
                if len(parts) >= 3:
                    return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])
        except: pass
        try: return float(val)
        except: return np.nan
    if isinstance(val, (int, float)):
        # Convert Excel day-fractions (e.g., 0.00038) to seconds
        return val * 86400 if val < 1.0 else val
    return np.nan

files = {
    2016: 'data/raw/GIC Monthly Data Collection 2016.xls',
    2017: 'data/raw/GIC Monthly Data Collection 2017.xls',
    2018: 'data/raw/GIC Monthly Data Collection 2018.xls',
    2020: 'data/raw/GIC Monthly Data Collection 2020.xls',
    2021: 'data/raw/GIC Monthly Data Collection 2021.xls'
}
metrics = ["Total no. of calls", "No. of calls successfully answered", 
           "No. of calls answered within 30 seconds", "Average waiting time in queue", 
           "No. of calls abandoned by the caller"]
month_map = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'June':6, 
             'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}

# Load Data
dfs = []
for year, filepath in files.items():
    try:
        df = pd.read_excel(filepath, header=4)
        df.rename(columns={df.columns[0]: 'Metric'}, inplace=True)
        df['Metric'] = df['Metric'].astype(str).str.strip()
        df = df[df['Metric'].isin(metrics)].copy()
        
        # Transpose
        df_t = df.set_index('Metric').transpose().reset_index().rename(columns={'index': 'Month'})
        df_t['Month'] = df_t['Month'].str.strip()
        df_t = df_t[df_t['Month'].isin(month_map.keys())].copy()
        df_t['Year'] = year
        df_t['Month_Num'] = df_t['Month'].map(month_map)
        
        for col in metrics:
            if col == "Average waiting time in queue":
                df_t[col] = df_t[col].apply(clean_waiting_time)
            else:
                df_t[col] = pd.to_numeric(df_t[col].astype(str).str.replace(',', ''), errors='coerce')
        dfs.append(df_t)
    except Exception as e: 
        print(f"Skipping {year}: {e}")

df_all = pd.concat(dfs, ignore_index=True)

In [2]:
# --- 2. Generate 2019 Data (Synthetic) ---
rows_2019 = []
for m_num in range(1, 13):
    m_name = [k for k,v in month_map.items() if v==m_num][0]
    row = {'Month': m_name, 'Year': 2019, 'Month_Num': m_num}
    for metric in metrics:
        v18 = df_all.loc[(df_all['Year']==2018) & (df_all['Month_Num']==m_num), metric].values
        v20 = df_all.loc[(df_all['Year']==2020) & (df_all['Month_Num']==m_num), metric].values
        v21 = df_all.loc[(df_all['Year']==2021) & (df_all['Month_Num']==m_num), metric].values
        
        val18 = v18[0] if len(v18)>0 else np.nan
        val20 = v20[0] if len(v20)>0 else np.nan
        val21 = v21[0] if len(v21)>0 else np.nan
        
        if pd.isna(val20):
            row[metric] = (val18 + val21) / 2 if pd.notna(val18) and pd.notna(val21) else val18
        else:
            row[metric] = (val18 + val20) / 2
    rows_2019.append(row)

df_hist = pd.concat([df_all, pd.DataFrame(rows_2019)], ignore_index=True)
df_hist['Date'] = pd.to_datetime(df_hist.assign(Day=1)[['Year','Month_Num','Day']].rename(columns={'Month_Num':'Month'}))


In [3]:
# Sort and Set Index
df_hist = df_hist.sort_values('Date').set_index('Date')

# 1. Set Frequency to Month Start ('MS')
df_hist = df_hist.asfreq('MS')
# 2. Interpolate time-based holes, then Backward Fill (for leading NaNs), then Forward Fill
df_hist[metrics] = df_hist[metrics].interpolate(method='time').fillna(method='bfill').fillna(method='ffill')




In [4]:
# --- 3. Forecast 2022-2025 ---
forecast_horizon = 46 # Jan 2022 to Oct 2025
future_dates = pd.date_range(start='2022-01-01', periods=forecast_horizon, freq='MS')

def forecast_series(series, steps):
    try:
        # Try standard Holt-Winters
        model = ExponentialSmoothing(series, seasonal_periods=12, trend='add', seasonal='add', damped_trend=True).fit()
    except:
        # Fallback if data is too flat/simple
        model = ExponentialSmoothing(series, seasonal_periods=12, trend='add', seasonal='add').fit()
    return model.forecast(steps)

# Generate Forecasts
pred_total = forecast_series(df_hist['Total no. of calls'], forecast_horizon)

# Ratios for consistency
ts_ans_rate = (df_hist['No. of calls successfully answered'] / df_hist['Total no. of calls']).clip(0, 1).fillna(method='bfill')
pred_ans_rate = forecast_series(ts_ans_rate, forecast_horizon).clip(0, 1)

ts_sl = (df_hist['No. of calls answered within 30 seconds'] / df_hist['No. of calls successfully answered']).clip(0, 1).fillna(method='bfill')
pred_sl = forecast_series(ts_sl, forecast_horizon).clip(0, 1)

# Wait time forecast (ensure no negatives)
pred_wait = forecast_series(df_hist['Average waiting time in queue'], forecast_horizon)
pred_wait[pred_wait < 0] = 0

# --- 4. Construct Final DataFrame ---
df_future = pd.DataFrame(index=future_dates)
df_future['Total no. of calls'] = pred_total.values
df_future['No. of calls successfully answered'] = (df_future['Total no. of calls'] * pred_ans_rate.values).astype(int)
df_future['No. of calls abandoned by the caller'] = (df_future['Total no. of calls'] - df_future['No. of calls successfully answered']).astype(int)
df_future['No. of calls answered within 30 seconds'] = (df_future['No. of calls successfully answered'] * pred_sl.values).astype(int)
df_future['Average waiting time in queue'] = pred_wait.values

# Metadata
df_future['Year'] = df_future.index.year
df_future['Month'] = df_future.index.month_name().str.slice(stop=3) # Jan, Feb
df_future['Month_Num'] = df_future.index.month


In [5]:
# Combine
df_final = pd.concat([df_hist.reset_index(), df_future.reset_index().rename(columns={'index':'Date'})], ignore_index=True)

# Save
df_final.to_csv('GIC_Full_Dataset_2016_2025.csv', index=False)
print("Success! Dataset Generated: GIC_Full_Dataset_2016_2025.csv")
print(df_final.tail())

Success! Dataset Generated: GIC_Full_Dataset_2016_2025.csv
          Date Month  Total no. of calls  No. of calls successfully answered  \
113 2025-06-01   Jun       121222.658866                             89066.0   
114 2025-07-01   Jul       127675.815264                             94283.0   
115 2025-08-01   Aug       134715.930550                             96270.0   
116 2025-09-01   Sep       130919.530451                             96079.0   
117 2025-10-01   Oct       134928.699115                             90492.0   

     No. of calls answered within 30 seconds  Average waiting time in queue  \
113                                  28519.0                     131.709244   
114                                  36220.0                     130.201308   
115                                  32598.0                     134.502375   
116                                  32947.0                     131.777886   
117                                  29996.0                     

In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Date                                     118 non-null    datetime64[ns]
 1   Month                                    118 non-null    object        
 2   Total no. of calls                       118 non-null    float64       
 3   No. of calls successfully answered       118 non-null    float64       
 4   No. of calls answered within 30 seconds  118 non-null    float64       
 5   Average waiting time in queue            118 non-null    float64       
 6   No. of calls abandoned by the caller     118 non-null    float64       
 7   Year                                     118 non-null    int64         
 8   Month_Num                                118 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(2), 